In [1]:
from IPython.display import clear_output
import mesa
import scipy.optimize as opt
import ipynb
import math
import numpy as np
from functools import reduce
clear_output(wait=True)
print("Everything imported!")


Everything imported!


In [10]:
from mesa import Agent
import random

class RandomWalker(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)

        self.pos = pos

    def move(self,target_pos,vision):
        heading=self.model.grid.get_heading(self.pos,target_pos)
        next_pos=tuple(np.array(heading)*(1+vision/2))
        self.model.grid.move_agent(self,next_pos)
        
    #generate a random position in a vision*vision square
    def _random_pos(self,vision):
        return tuple(np.array(self.pos)+(np.random.rand(2)-0.5)*vision)
    
    def random_move(self,vision):
        self.model.grid.move_agent(self,self._random_pos(vision))
        

In [27]:
class Individual(RandomWalker):
    def __init__(self, unique_id, model, pos, 
                 sex, income, level, generation,clan,generation_in_clan):
        super().__init__(unique_id, model, pos)
        self.sex=sex  #False:male, True:Female
        self.income=income
        self.marriage=False 
        self.investment=(0,0) #(male_investment, female_investment)
        self.clan=clan # family id
        self.generation_in_clan=generation_in_clan # generation in a certain family
        self.children=(0,0)  #(male_count, female_count)
        self.generation=generation  #current generation in population
        self.level=level #0->4 low->high
        self.vision=self.model.vision
        self.current_step=0

    def _find_nearest(self,pos,neighbors):
        
        # find opposite gender satisfying following conditions
        op_gender=list(filter(lambda n:(n.sex^self.sex) 
                                and (not n.marriage)
                                and n.generation==self.generation #only find hetersexual at the same generation
                                and n.clan != self.clan #matting in family is forbidden
                                and n.level==self.level #find 
                                ,neighbors))
        incomes=list(map(lambda n:n.income,op_gender))
        if len(incomes)==0:
            return None
        i=np.argmax(incomes) 
        return op_gender[i]
    
    
    # reproduction plan
    def get_plan(self,neighbors):
        male_u, female_u=self.model.get_utility()
        reproduce_plan=[]
        plan_utility=[]
        n_i=self.get_neighbor_investment(neighbors) 
        
        #compute competitiveness independently
        #formula: current_investment / average_investment_of_neighbors (including self)
        compet_male=lambda c,x:1 if((n_i[2]+x[0])==0 or n_i[0]+c[0]==0) else x[0]/((n_i[2]+x[0])/(n_i[0]+c[0]))
        compet_female=lambda c,x:1 if((n_i[3]+x[1])==0 or n_i[1]+c[1]==0) else x[1]/((n_i[3]+x[1])/(n_i[1]+c[1]))
        
        #compute investment plan for all family combinations
        for c in self.model.family_set:
            target=lambda x:-(c[0]*x[0]*male_u*compet_male(c,x)+c[1]*x[1]*female_u*compet_female(c,x)) #explanation of the formula is in the shared document
            condition=lambda x:self.income-c[0]*x[0]-c[1]*x[1] #total investment is less than parents' total income
            cons=[{"type":"ineq",'fun':condition}]
            bnds=((0.1,None),(0.1,None))  # minimum investment for each kid, adjustable
            res = opt.minimize(target, (1, 1), method='trust-constr', bounds=bnds, constraints=cons,options={'maxiter':10})
            #this condition statement handles exceptions caused by numerical algorithms
            if(res.x[0]<0 or res.x[1]<0):
                res.x=(0,0)
            #structure of plan: ((male,famale),(invest_male, invest_female),(compet_male,compet_female))
            mc=compet_male(c,res.x)
            fc=compet_female(c,res.x)
            reproduce_plan.append((c,(res.x[0],res.x[1]),(mc,fc)))
            plan_utility.append(-target(res.x)) #record function value of 'target', generating distribution
        plan_utility=np.array(plan_utility)
        dist=plan_utility/sum(plan_utility)         
        plan=np.random.choice(list(range(len(dist))),size=1,p=dist)[0] #choose a plan according to target values
        return reproduce_plan[plan]
        
    def get_neighbor_investment(self, neighbors):
        male_count,male_sum,female_count,female_sum=0,0,0,0
        for n in neighbors:
            if((not n.marriage) or (n.generation!=self.generation)):
                continue
            male_count+=n.children[0]
            male_sum+=n.investment[0]*n.children[0]
            female_count+=n.children[1]
            female_sum+=n.investment[1]*n.children[1]
        return male_count,female_count,male_sum,female_sum
            
    def reproduce(self,neighbors):
        children,investment,compet=self.get_plan(neighbors)
        utility=self.model.utilities
        self.investment=investment
        self.children=children
        mc,fc=1,1
        if(self.generation>0): #the first generation has no married neighbors
            mc,fc=compet
        
        expected_income=self.model.level_income[self.level] #expected income is defined by the average income of the parents' level
        male_income=expected_income*mc*utility[0]
        female_income=expected_income*fc*utility[1]
        male_level=self.level
        female_level=self.level
        #if the income value is larger than average income of a higher level, the child upgrades level
        for i in range(self.level+1,5):
            if(male_income>self.model.level_income[i]):
                male_level=i
            if(female_income>self.model.level_income[i]):
                female_level=i
        
        
        #if the income value is less than average income of a lower level, the child upgrades level
        for i in range(0,self.level):
            if(male_income<self.model.level_income[i]):
                male_level=i
            if(female_income<self.model.level_income[i]):
                female_level=i
                
        next_clan=self.clan #record family identity
        next_g=self.generation_in_clan+1
        if(next_g>3): #if the kid is far from the his/her ancestor, mark him/her as a new family
            next_clan=None
            next_g=0
        
        for i in range(children[0]):
            self.model.new_agent(Male,self.pos,male_income,male_level,self.generation+1,next_clan,next_g)
        for i in range(children[1]):
            self.model.new_agent(Female,self.pos,female_income,female_level,self.generation+1,next_clan,next_g)
    
    
    def step(self):
        self.current_step+=1
        #define agent's life
        if(self.current_step>self.model.max_step):
            self.model.remove_agent(self)
            return 
        if(self.marriage):
            return
        
        neighbors=self.model.grid.get_neighbors(self.pos,self.vision,False)
        candidate=self._find_nearest(self.pos,neighbors)
        if(candidate is None):
            self.random_move(self.vision)
            return
        if(random.random()<self.marriage_chance(candidate)):
            self.income+=candidate.income  #merge income
            self.marriage=True
            self.reproduce(neighbors)
            self.model.remove_agent(candidate) #remove the candidate because one representitive agent is enough for a family
        else:
            self.random_move(self.vision)


class Male(Individual):
    def __init__(self,unique_id, model, pos, 
                 income, level, generation=1,clan=None,generation_in_clan=0):
        super().__init__(unique_id, model, pos, 
                 False, income, level, generation,clan,generation_in_clan)
                
    def marriage_chance(self,candidate):
        return (self.level==candidate.level)
    
        
class Female(Individual):
    def __init__(self,unique_id, model, pos, 
                 income, level, generation=1,clan=None,generation_in_clan=0):
        super().__init__(unique_id, model, pos, 
                 True, income, level, generation,clan,generation_in_clan)
        
    
    def marriage_chance(self,candidate):
        return (self.level==candidate.level)


In [30]:
from mesa import Model
from mesa.space import ContinuousSpace
from mesa.datacollection import DataCollector
from mesa.time import RandomActivation
import itertools as it
def compute_male_income(model):
    male_income=0
    male_count=0
    for agent in model.schedule_population.agents:
        if(not agent.sex and (not agent.marriage)):
            male_income+=agent.income
            male_count+=1
    if(male_count==0):
        return 0
    return male_income/male_count

def compute_female_income(model):
    female_income=0
    female_count=0
    for agent in model.schedule_population.agents:
        if(agent.sex and (not agent.marriage)):
            female_income+=agent.income
            female_count+=1
    if(female_count==0):
        return 0
    return female_income/female_count

def male_at_birth(model):
    male_count=0
    for agent in model.schedule_population.agents:
        male_count+=agent.children[0]
    return male_count

def female_at_birth(model):
    female_count=0
    for agent in model.schedule_population.agents:
        female_count+=agent.children[1]
    return female_count

def male_investment(model):
    m_count=0
    inv=0
    for agent in model.schedule_population.agents:
        if(agent.marriage):
            if(agent.children[0]>0):
                m_count+=agent.children[0]
                inv+=agent.investment[0]*agent.children[0]
    if(m_count==0):
        return 0
    return inv/m_count
    
def female_investment(model):
    f_count=0
    inv=0
    for agent in model.schedule_population.agents:
        if(agent.marriage):
            if(agent.children[1]>0):
                f_count+=agent.children[1]
                inv+=agent.investment[1]*agent.children[1]
    if(f_count==0):
        return 0
    return inv/f_count

class SexDistortion(Model):
    
    def __init__(self, height=30, width=30,
                 base_population=1000,planning=3,
                 sex_ratio=110/100,
                 social_level=5,
                 income_bias=0.8,  # income bias, p percent people hold (1-p) percent wealth, adjustable
                 vision=2, initial_utility=(1,0.8)):

        super().__init__()
        
        self.sex_ratio=sex_ratio
        self.social_level=social_level
        self.income_bias=income_bias
        self.height = height
        self.width = width
        self.vision=vision
        self.schedule_population=RandomActivation(self)
        self.max_step=3
        self.grid = ContinuousSpace(self.width, self.height, torus=True)
        self.datacollector = DataCollector(
             model_reporters={"male_income": compute_male_income,
                              "female_income":compute_female_income,
                              "male_count":male_at_birth,
                              "female_count":female_at_birth,
                              "male_investment":male_investment,
                              "female_investment":female_investment
                             })
        self.utility_counter=-1
        self.utilities=initial_utility
        self.family_set=set()
        i=list(range(planning+1))+list(range(planning+1))
        p=it.permutations(i,2)
        for c in p:
            if(c[0]+c[1]<=planning):
                self.family_set.add(c)
        self.current_step=0

        self.init_population_v1(base_population)
        self.level_income={}
        self.level_count={}

        # This is required for the datacollector to work
        self.running = True
        self.datacollector.collect(self)
        
#     def init_population(self, urban_female):
#         urban_male=math.floor(urban_female*self.urban_sex_ratio)
#         country_female=math.floor(urban_female/self.urban_country_ratio)
#         country_male=math.floor(country_female*self.country_sex_ratio)
#         country_male_income=5
#         urban_male_income=15
#         u=self.utilities[1]/self.utilities[0]
#         for i in range(urban_male):
#             x = random.random()*(self.width)
#             y = random.random()*(self.height)
#             self.new_agent(Male,(x,y),urban_male_income,'urban')
#         for i in range(urban_female):
#             x = random.random()*(self.width)
#             y = random.random()*(self.height)
#             self.new_agent(Female,(x,y),urban_male_income*u,'urban')
#         for i in range(country_male):
#             x = random.random()*(self.width)
#             y = random.random()*(self.height)
#             self.new_agent(Male,(x,y),country_male_income,'country')
#         for i in range(country_female):
#             x = random.random()*(self.width)
#             y = random.random()*(self.height)
#             self.new_agent(Female,(x,y),country_male_income*u,'country')
            
            
    def init_population_v1(self, total):
        
        u=self.utilities[1]/self.utilities[0]
        
        #compute population detail by sex_ratio
        male_count=total*self.sex_ratio/(self.sex_ratio+1)
        female_count=total-male_count
        
        #approximate lorenz curve by staged poisson distribution
        X_male=np.append(np.random.poisson(lam=5, size=math.ceil(male_count*self.income_bias)), 
              np.random.poisson(lam=30, size=math.ceil(male_count*(1-self.income_bias))))
        X_male=np.sort(X_male)
        slot_width=len(X_male)//self.social_level
        for i in range(len(X_male)):
            x = random.random()*(self.width)
            y = random.random()*(self.height)
            self.new_agent(Male,(x,y),X_male[i],i//slot_width)
            
        
        X_female=np.append(np.random.poisson(lam=5*u, size=math.ceil(female_count*self.income_bias)), 
              np.random.poisson(lam=30*u, size=math.ceil(female_count*(1-self.income_bias))))
        X_female=np.sort(X_female)
        slot_width=len(X_female)//self.social_level
        for i in range(len(X_female)):
            x = random.random()*(self.width)
            y = random.random()*(self.height)
            self.new_agent(Female,(x,y),X_female[i],i//slot_width)
    
    #pos, income, residence, generation=1,clan=None,generation_in_clan=0, utility=1.0)
    def new_agent(self, agent_type, pos,income,level,generation=1,clan=None,generation_in_clan=0):
        '''
        Method that creates a new agent, and adds it to the correct scheduler.
        '''
        i=self.next_id()
        if(clan is None):
            clan=i
        agent = agent_type(i, self, pos,income,level,generation,clan,generation_in_clan)

        self.grid.place_agent(agent, pos)
        self.schedule_population.add(agent)

    def remove_agent(self, agent):
        '''
        Method that removes an agent from the grid and the correct scheduler.
        '''
        self.grid.remove_agent(agent)
        self.schedule_population.remove(agent)

    def step(self):
        '''
        Method that calls the step method for each of the sheep, and then for each of the wolves.
        '''
        self.schedule_population.step()

        # Save the statistics
        self.datacollector.collect(self)
    
    #refreshing income ratio
    def get_utility(self):
        if(self.current_step>self.utility_counter):
            data = model.datacollector.get_model_vars_dataframe()[-1:]
            print(data)
            a,b=(data["male_income"].to_numpy()[0],data["female_income"].to_numpy()[0])
            self.utilities=(1,b/a)
            self.utility_counter=self.current_step
        return self.utilities
    
    def run_model(self, step_count=15):
        '''
        Method that runs the model for a specific amount of steps.
        '''
        for i in range(step_count):
            print(i)
            self.current_step=i
            self.stat_level_income()
            print(self.level_income)
            print(self.level_count)
            self.step()
        
    def stat_level_income(self):
        level_count={}
        level_sum={}
        for agent in self.schedule_population.agents:
            if(agent.marriage):
                continue
            level_count[agent.level]=level_count.get(agent.level,0)+1
            level_sum[agent.level]=level_sum.get(agent.level,0)+agent.income
        for i in range(5):
            self.level_income[i]=level_sum.get(i,0)/level_count.get(i,1)
            self.level_count[i]=level_count.get(i,0)

In [31]:

%matplotlib inline
    
model = SexDistortion()
model.run_model()

data = model.datacollector.get_model_vars_dataframe()



0
{0: 1.96, 1: 3.79, 2: 5.08, 3: 7.23, 4: 26.335}
{0: 200, 1: 200, 2: 200, 3: 200, 4: 200}
   male_income  female_income  male_count  female_count  male_investment  \
0     9.929524       7.832285           0             0                0   

   female_investment  
0                  0  


/Users/sunweber/.local/share/virtualenvs/abm-assignments-nFGxnKtf/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


1
{0: 1.4754116781854079, 1: 3.276965956226808, 2: 4.822040000237823, 3: 11.212785734011886, 4: 51.35663269652996}
{0: 208, 1: 146, 2: 181, 3: 349, 4: 214}
   male_income  female_income  male_count  female_count  male_investment  \
1    15.946037      14.106063         397           333         7.189201   

   female_investment  
1           7.788833  


/Users/sunweber/.local/share/virtualenvs/abm-assignments-nFGxnKtf/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


2
{0: 1.4168087679846062, 1: 3.324440055831232, 2: 5.189169678202901, 3: 16.61906876595899, 4: 78.67961438259735}
{0: 199, 1: 162, 2: 192, 3: 389, 4: 228}
   male_income  female_income  male_count  female_count  male_investment  \
2    23.996486      20.614037         597           557         9.833066   

   female_investment  
2           9.274152  


...

8
{0: 1.5304314110601818, 1: 3.9972062676095077, 2: 13.463112171404397, 3: 57.64537659582739, 4: 385.8311936699239}
{0: 24, 1: 12, 2: 26, 3: 52, 4: 21}
   male_income  female_income  male_count  female_count  male_investment  \
8   115.276176       45.83598          50            44        46.803193   

   female_investment  
8          56.930077  


/Users/sunweber/.local/share/virtualenvs/abm-assignments-nFGxnKtf/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


9
{0: 1.798989376009307, 1: 4.4513051795834215, 2: 16.67413524896791, 3: 62.46123265632034, 4: 562.4504680248336}
{0: 21, 1: 7, 2: 19, 3: 35, 4: 15}
   male_income  female_income  male_count  female_count  male_investment  \
9   156.016181      47.465577          39            22        85.613352   

   female_investment  
9          54.498649  


/Users/sunweber/.local/share/virtualenvs/abm-assignments-nFGxnKtf/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


10
{0: 1.9581821037810445, 1: 3.8982993966402533, 2: 16.241426501738218, 3: 84.5354473879465, 4: 622.8834285746461}
{0: 10, 1: 4, 2: 8, 3: 17, 4: 10}
11
{0: 2.4544615494897117, 1: 3.8306538320340855, 2: 0.0, 3: 142.8516530669247, 4: 660.1640614444173}
{0: 6, 1: 3, 2: 0, 3: 5, 4: 8}
12
{0: 2.5703959316851703, 1: 0.0, 2: 0.0, 3: 151.9982175061561, 4: 745.3278251655966}
{0: 5, 1: 0, 2: 0, 3: 4, 4: 6}
13
{0: 0.0, 1: 0.0, 2: 0.0, 3: 187.383697968961, 4: 0.0}
{0: 0, 1: 0, 2: 0, 3: 3, 4: 0}
14
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0}
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0}


In [32]:
print(data)
print(model.level_income)
print(model.level_count)

    male_income  female_income  male_count  female_count  male_investment  \
0      9.929524       7.832285           0             0         0.000000   
1     15.946037      14.106063         397           333         7.189201   
2     23.996486      20.614037         597           557         9.833066   
3     29.876240      24.383893         729           666        12.356706   
4     41.110106      30.402214         375           365        22.836043   
5     65.735482      40.940563         191           168        30.322320   
6     80.780533      53.082895         111            92        26.492770   
7     99.109653      48.506015          70            71        38.166556   
8    115.276176      45.835980          50            44        46.803193   
9    156.016181      47.465577          39            22        85.613352   
10   203.283250      61.294189          23            11        91.896341   
11   315.282905     132.396022          12             5       119.007326   